<a href="https://colab.research.google.com/github/ritwiktiwari/covid-website/blob/master/CHEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"shachisharma8","key":"783cb594e5e4926126e74518acde2c8a"}'}

In [87]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [88]:
!kaggle datasets download -d nih-chest-xrays/sample


sample.zip: Skipping, found more recently modified local copy (use --force to force download)


In [89]:
!unzip sample.zip



Archive:  sample.zip
replace sample/images/00000013_005.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [90]:
import numpy as np
import pandas as pd
import os
from glob import glob
import matplotlib.pyplot as plt



In [155]:
labels=pd.read_csv('sample_labels.csv', dtype=str)

image_path={os.path.basename(x): x for x in glob(os.path.join('sample/images/*png'))}

print('Images found:', len(image_path), 'Total headers:',labels.shape[0])

labels['path']=labels['Image Index'].map(image_path.get)


disease_vec = ["Atelectasis", "Cardiomegaly", "Consolidation", "Edema", "Effusion", "Emphysema", "Fibrosis", "Hernia", "Infiltration", "Mass", "No Finding", "Nodule", "Pleural_Thickening", "Pneumonia", "Pneumothorax"]


label_counts = labels['Finding Labels'].value_counts()[:17]

labels.head()



Images found: 5606 Total headers: 5606


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,path
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,005,00000013,060Y,M,AP,3056,2544,0.139,0.139,sample/images/00000013_005.png
1,00000013_026.png,Cardiomegaly|Emphysema,026,00000013,057Y,M,AP,2500,2048,0.168,0.168,sample/images/00000013_026.png
2,00000017_001.png,No Finding,001,00000017,077Y,M,AP,2500,2048,0.168,0.168,sample/images/00000017_001.png
3,00000030_001.png,Atelectasis,001,00000030,079Y,M,PA,2992,2991,0.143,0.143,sample/images/00000030_001.png
4,00000032_001.png,Cardiomegaly|Edema|Effusion,001,00000032,055Y,F,AP,2500,2048,0.168,0.168,sample/images/00000032_001.png


In [156]:
'''
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)
'''
from itertools import chain
all_labels = np.unique(list(chain(*labels['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        labels[c_label] = labels['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)

All Labels (15): ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [157]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(labels,
                                   test_size = 0.25,
                                   random_state = 500,
                                   stratify = labels['Finding Labels'].map(lambda x: x[:4]))
print('train', train_df.shape[0], 'validation', valid_df.shape[0])




from keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = (128,128)

train 4204 validation 1402


In [158]:
datagen = ImageDataGenerator(samplewise_center=True,
                              samplewise_std_normalization=True,
                              horizontal_flip = True,
                              vertical_flip = False,
                             rescale = 1./255,
                              height_shift_range= 0.05,
                              width_shift_range=0.1,
                              rotation_range=5,
                              shear_range = 0.1,
                              fill_mode = 'reflect',
                              zoom_range=0.15)






train_gen = datagen.flow_from_dataframe(dataframe= train_df,
                                        directory='./sample/images/',
                                        x_col='Image Index',
                                        y_col = disease_vec,
                                        target_size = IMG_SIZE,
                                        color_mode = 'grayscale',
                                        class_mode='raw',
                                        shuffle=True,
                                        batch_size = 32)

valid_gen = datagen.flow_from_dataframe(dataframe= valid_df,
                                        directory='./sample/images/',
                                        x_col = 'Image Index',
                                        y_col = disease_vec,
                                        class_mode='raw',
                            target_size = IMG_SIZE,
                             color_mode = 'grayscale',
                            batch_size = 256)




Found 4204 validated image filenames.
Found 1402 validated image filenames.


In [160]:

import tensorflow as tf
from keras.applications.mobilenet import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.models import Sequential
base_mobilenet_model = MobileNet(input_shape = (128,128,1), include_top = False, weights = None)
multi_disease_model = Sequential()
multi_disease_model.add(base_mobilenet_model)
multi_disease_model.add(GlobalAveragePooling2D())
multi_disease_model.add(Dropout(0.5))
multi_disease_model.add(Dense(512))
multi_disease_model.add(Dropout(0.5))
multi_disease_model.add(Dense(len(all_labels), activation = 'sigmoid'))
multi_disease_model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy', 'mae'])
multi_disease_model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_128 (Model)   (None, 4, 4, 1024)        3228288   
_________________________________________________________________
global_average_pooling2d_6 ( (None, 1024)              0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 15)                7695      
Total params: 3,760,783
Trainable params: 3,738,895
Non-trainable params: 21,888
_______________________________________

In [ ]:


from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('xray_class')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1,
                             save_best_only=True, mode='min', save_weights_only = True)

early = EarlyStopping(monitor="val_loss",
                      mode="min",
                      patience=3)
callbacks_list = [checkpoint, early]

history = multi_disease_model.fit_generator(generator=train_gen,
                    steps_per_epoch=50,
                    validation_data=valid_gen,
                    validation_steps=50,
                    epochs=10)


Epoch 1/10
49/50 [============================>.] - ETA: 3s - loss: 0.3492 - binary_accuracy: 0.8973 - mae: 0.1323